<a href="https://colab.research.google.com/github/Ibsen-Gomes/Deep_Learning_Convolutional_Neural_Network/blob/main/Part_2_Convolutional_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Redes neurais convolucionais para classificação de gatos e cachorros.

In [11]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from torchvision import datasets
import torchvision.transforms as transforms
torch.__version__

'2.0.1+cu118'

In [12]:
torch.manual_seed(42)

### Construção do modelo:

In [13]:
classificador = nn.Sequential(nn.Conv2d(in_channels=3, # imagems coloridas = 3 canais RGB
                                        out_channels=32, # 32 filtros
                                        kernel_size=3), # matrix 3x3
                              nn.ReLU(),
                              nn.BatchNorm2d(num_features=32),  # normalização nas camadas ocultas
                              nn.MaxPool2d(kernel_size=2), # pooling com matrix 2x2
                              nn.Conv2d(32, 32, 3), #chegando 32 filtros + gerando outros 32 filtros + matrix 3x3 (kernel)
                              nn.ReLU(),
                              nn.BatchNorm2d(num_features=32),
                              nn.MaxPool2d(kernel_size=2),
                              nn.Flatten(), #transformnado a imagem em um vetor pra a camada densa

                              # Camada densa:

                              ### 1° camada:
                              nn.Linear(in_features=14*14*32, out_features=128), # 14x14x32
                              nn.ReLU(),
                              nn.Dropout(0.2),

                              ### 2° camada:
                              nn.Linear(128, 128),
                              nn.ReLU(),
                              nn.Dropout(0.2),

                              # saída:
                              nn.Linear(128, 1),#return 1 pq é um probelma de classificação binária
                              nn.Sigmoid() # p/ retornar o valor da robabilidade (cahorro ou gato)
                              )




In [14]:
criterion = nn.BCELoss() # binary cross entropy
optimizer = optim.Adam(classificador.parameters())

### Base de dados:

In [15]:
# Criando um gdrive:
from google.colab import drive
drive.mount('/content/dgrive')

Drive already mounted at /content/dgrive; to attempt to forcibly remount, call drive.mount("/content/dgrive", force_remount=True).


In [16]:
import zipfile

In [18]:
# descompactando pasta com os aruivos de treinamento e teste:

path = '/content/dgrive/MyDrive/Deep_learning/dataset.zip'
zip_object = zipfile.ZipFile(file=path, mode='r')
zip_object.extractall('./')
zip_object.close()

In [19]:
data_dir_train = '/content/dataset/training_set'
data_dir_test  = '/content/dataset/test_set'

In [21]:
# Transformador para a base de dados de treinamento:

transform_train = transforms.Compose(
    [transforms.Resize([64, 64]),
     transforms.RandomHorizontalFlip(),
     transforms.RandomAffine(degrees=7, translate=(0, 0.07,), shear=0.2, scale=(1, 1.2)),
     transforms.ToTensor()
     ]

)

In [22]:
# Transformador para a base de dados de teste:

transform_test = transforms.Compose(
    [transforms.Resize([64, 64]),
     transforms.ToTensor()
     ]

)

In [23]:
# Criação da base de dados:

train_dataset = datasets.ImageFolder(data_dir_train, transform=transform_train)

In [24]:
# Mostra as transformações setadas:
train_dataset

Dataset ImageFolder
    Number of datapoints: 4000
    Root location: /content/dataset/training_set
    StandardTransform
Transform: Compose(
               Resize(size=[64, 64], interpolation=bilinear, max_size=None, antialias=warn)
               RandomHorizontalFlip(p=0.5)
               RandomAffine(degrees=[-7.0, 7.0], translate=(0, 0.07), scale=(1, 1.2), shear=[-0.2, 0.2])
               ToTensor()
           )

In [26]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)

In [27]:
# Agora para teste:

test_dataset = datasets.ImageFolder(data_dir_test, transform=transform_test)

In [28]:
# Mostra as transformações setadas:
test_dataset

Dataset ImageFolder
    Number of datapoints: 1000
    Root location: /content/dataset/test_set
    StandardTransform
Transform: Compose(
               Resize(size=[64, 64], interpolation=bilinear, max_size=None, antialias=warn)
               ToTensor()
           )

In [29]:
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=True)

### Treinamento: